# Methanol-Synthese
Bedingungen: 

T=493;15K

p=50 bar

## Bilanzen
Stoffbilanzen

$\dot n_i = \dot n_{i, 0} - \sum_{j}{\nu_ij \xi_j}$ 

Energiebilanz

$0 = \dot Q + \sum\limits_{j}{\xi_j \Delta H_j}$

## Gleichgewichts-Konstanten

$exp \left(- \frac{\Delta G_i}{R T} \right) = K_p K_{\phi^{eq}} = K_x \prod_i \left( \frac{p}{p^0}\right)^{\nu_i}  K_{\phi^{eq}}$

Idealer Gas, $K_{\phi^{eq}}=1$ 

Van't Hoff, $\frac{d ln K}{dT} = -\frac{\Delta H}{R T^2}$

~ Geringe Veränderung der Reaktionsenthalpie mit der Temperatur: $\frac{d ln K}{dT} = -\frac{\Delta H}{R T^2} \sim \Rightarrow ln \left(\frac{K}{K'} \right) = -\frac{\Delta H^0}{R}\left(\frac{1}{T} - \frac{1}{T'} \right)$

$K_{(493,15K)} = K_{(298,15K)} \times exp \left[-\frac{\Delta H^0}{R}\left(\frac{1}{493,15 K} - \frac{1}{298,15 K} \right)\right] = K_x \prod\limits_{i}{\left( \frac{p}{p^0}\right)^{\nu_i} }= K_x \left( \frac{p}{p^0}\right)^{\sum\limits_{i}{\nu_i}}$ 

In [182]:
from scipy import optimize
import numpy as np

p = 50.0 # bar
temp = 273.15 + 220. # K
r = 8.314 # J/(mol K)

namen = ['CO', 'H2', 'CO2', 'H2O', 'CH3OH']

n0co = 50.
n0h2 = 100.
n0co2 = 10.
n0h2o = 0.
n0ch3oh = 0.

delta_h_1_298 = (-(-110.53) -2*0 + -200.94)*1000
delta_h_2_298 = (-(-393.50) -3*0 + -200.94 + -241.83)*1000
delta_h_3_298 = (-(-110.53) -(-241.83) + -393.50 + 0)*1000

delta_g_1_298 = (-(-137.20) -2*0 + -162.32)*1000
delta_g_2_298 = (-(-394.36) -3*0 + -162.32 + -228.62)*1000
delta_g_3_298 = (-(-137.20) -(-228.62) + -394.36 + 0)*1000

k1_298 = np.exp(-delta_g_1_298/(r*temp))
k2_298 = np.exp(-delta_g_2_298/(r*temp))
k3_298 = np.exp(-delta_g_3_298/(r*temp))

# Angenommen, dass Delta_H fast im Interval zwischen 298.15K und 443.15K 
# unabhängig von der Temperatur sei...
k1_493 = k1_298 * np.exp(-delta_h_1_298/8.314*( 1/493.15 - 1/298.15 ))
k2_493 = k2_298 * np.exp(-delta_h_2_298/8.314*( 1/493.15 - 1/298.15 ))
k3_493 = k3_298 * np.exp(-delta_h_3_298/8.314*( 1/493.15 - 1/298.15 ))

for i, f in enumerate([delta_h_1_298, delta_h_2_298, delta_h_3_298]):
    print('Delta H_' + str(i+1) + '(298.15K)=' + str(f/1000.) + 'kJ/mol')

print('\n')
for i, f in enumerate([k1_493, k2_493, k3_493]):
    print('K' + str(i+1) + '(493K)=' + str(f))
print('\n')
    
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nch3oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nco + n0co - xi1
    f2 = -nh2  + n0h2 -2*xi1
    f3 = -nch3oh + n0ch3oh +xi1
    f4 = -k1_443 * (nco * nh2) + nch3oh * (p/1.)**-2
    
    return [f1, f2, f3, f4]

n0 = np.array([n0co, n0h2, n0ch3oh])
x0 = np.append(n0, [0])
sol = optimize.root(fun, x0)
print(sol)
print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für nur einzige Reaktion (ohne CO2):\n')
for i, f in enumerate(sol.x[:2]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
print('n_' + namen[-1] + '= ' + str(sol.x[2]) + ' mol')

# Lösung des einfacheren Falls in schwierigerem Fall einwenden.
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nco2 = x_vec[2]
    nh2o = x_vec[3]
    nch3oh = x_vec[4]
    xi1 = x_vec[5]
    xi2 = x_vec[6]
    xi3 = x_vec[7]
    
    f1 = -nco + n0co - xi1 +0 -xi3
    f2 = -nh2  + n0h2 -2*xi1 -3*xi2 +xi3
    f3 = -nco2 + n0co2 +0 -xi2 +xi3
    f4 = -nh2o + n0h2o +0 +xi2 -xi3
    f5 = -nch3oh + n0ch3oh +xi1 +xi2 -0
    f6 = -k1_443 * (nco * nh2) + nch3oh * (p/1.)**-2
    f7 = -k2_443 * (nco2 * nh2**2) + nch3oh * nh2o * (p/1.)**-2
    f8 = -k3_443 * (nco * nh2o) + nco2 * nh2 * (p/1.)**0
    
    return [f1, f2, f3, f4, f5, f6, f7, f8]

ne = np.array([n0co, n0h2, n0co2, n0h2o, n0ch3oh])

n0 = np.array([sol.x[0], sol.x[1], n0co2, 0, sol.x[2]])
x0 = np.append(n0, [sol.x[3], 0, 0])
#n0 = np.array([1.79111141, 3.58222282, 10., 0, 48.20888859])
#x0 = np.append(n0, [48.20888859, 0, 0])
sol = optimize.root(fun, x0)

nuij = np.array([[-1, -2, 0, 0, +1] ,
                 [0, -3, -1, +1, +1], 
                 [-1, +1, +1, -1, 0]]).T

f_final = - sol.x[:5].reshape([5,1]) + ne.reshape([5,1]) + np.dot(nuij , sol.x[5:].reshape([3,1]))

print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für alle drei Reaktionen, mit CO2:\n')
for i, f in enumerate(sol.x[:5]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
    
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

Delta H_1(298.15K)=-90.41kJ/mol
Delta H_2(298.15K)=-49.27kJ/mol
Delta H_3(298.15K)=-41.14kJ/mol


K1(493K)=0.000249687316504
K2(493K)=0.000167654187465
K3(493K)=1.48929961297


    fjac: array([[ -9.70445787e-01,  -3.58426591e-13,   2.55451912e-17,
         -2.41319240e-01],
       [  1.23870627e-02,  -9.98681716e-01,   4.89840315e-17,
         -4.98135698e-02],
       [  1.10084403e-02,   2.34467758e-03,  -9.98956217e-01,
         -4.42695515e-02],
       [  2.40749559e-01,   5.12770275e-02,   4.56779647e-02,
         -9.68154863e-01]])
     fun: array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -9.07156295e-14])
 message: 'The solution converged.'
    nfev: 17
     qtf: array([  7.99910767e-10,   1.65119080e-10,   1.46742095e-10,
         3.20918258e-09])
       r: array([ 1.03045427,  0.01278115,  0.01138555,  1.03933814,  1.00132002,
        0.00235023,  1.99919726,  1.00104487, -1.00201584, -0.02123464])
  status: 1
 success: True
       x: array([  1.79111141, 